In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy import sparse
#from scipy import ndimage
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [ ]:
g_data_name = "L64" # L64, zen64, syou64, dance64
g_d = 3 # 圧縮率，1/g_dに圧縮する．
g_noise_fac = 0.05 #ノイズ値
g_alpha1 = 10**(-5) # hyperparameter for Lasso
g_alpha2 = 1 # hyperparmeter for Ridge regression
g_fit_intercept = False

In [ ]:
G_METADATA = {"outputdir": "image_executed", "prefix": "tomography", 
              "dataname":g_data_name, "compression": g_d, "noise_fac": g_noise_fac }

In [ ]:
# このセルはscikit learnのコードです．文献[3]

def _weights(x, dx=1, orig=0):
    x = np.ravel(x)
    floor_x = np.floor((x - orig) / dx).astype(np.int64)
    alpha = (x - orig - floor_x * dx) / dx
    return np.hstack((floor_x, floor_x + 1)), np.hstack((1 - alpha, alpha))


def _generate_center_coordinates(l_x):
    X, Y = np.mgrid[:l_x, :l_x].astype(np.float64)
    center = l_x / 2.
    X += 0.5 - center
    Y += 0.5 - center
    return X, Y


def build_projection_operator(l_x, n_dir):
    """ Compute the tomography design matrix.

    Parameters
    ----------

    l_x : int
        linear size of image array

    n_dir : int
        number of angles at which projections are acquired.

    Returns
    -------
    p : sparse matrix of shape (n_dir l_x, l_x**2)
    """
    X, Y = _generate_center_coordinates(l_x)
    angles = np.linspace(0, np.pi, n_dir, endpoint=False)
    data_inds, weights, camera_inds = [], [], []
    data_unravel_indices = np.arange(l_x ** 2)
    data_unravel_indices = np.hstack((data_unravel_indices,
                                      data_unravel_indices))
    for i, angle in enumerate(angles):
        Xrot = np.cos(angle) * X - np.sin(angle) * Y

        inds, w = _weights(Xrot, dx=1, orig=X.min())
        mask = np.logical_and(inds >= 0, inds < l_x)

        weights += list(w[mask])
        camera_inds += list(inds[mask] + i * l_x)
        data_inds += list(data_unravel_indices[mask])
    proj_operator = sparse.coo_matrix((weights, (camera_inds, data_inds)))
    return proj_operator

In [ ]:
def get_data(dataname):
    """load data

    Args:
        dataname (str): データ名.
        
    Returns:
        np.array: imageデータ
    """
    # data directory
    prefix = "../data/font"
    filename = dataname + ".csv"

    data = np.loadtxt(os.path.join(prefix,filename), delimiter=",")
    return data

g_data = get_data(g_data_name)

In [ ]:
def normalize_data(data):
    """画像値の規格化

    Args:
        data (np.array): image

    Returns:
        np.array: 規格化された画像
    """
    m1 = data.ravel().min()
    m2 = data.ravel().max()
    """
    normalize data
    """
    data = (data-m1)/(m2-m1)
    return data
# the range of data is now [0:1]


g_w = normalize_data(g_data)
print(g_w.shape)

In [ ]:
plt.imshow(g_data, cmap=plt.cm.gray, interpolation='nearest')


In [ ]:
plt.hist(g_data.ravel(), bins=100)

In [ ]:

# このセルはsklearnのコードです．

def make_X(l, d):
    """
    (l,l)のサイズの画像からl*l/dしたトモグラフィ像を得る．

    """
    N = l*l
    print("N=", l*l)
    print("P=", l*l//d)
    print("P/N= {}%".format((l//d)/l*100))
    """
    construcut a tomography image
    l//d = int(l/d) = the number of angles 
    """
    X = build_projection_operator(l, l//d)
    print("X.shape=", X.shape)
    return X


g_X = make_X(g_w.shape[0], g_d)

In [ ]:
type(g_X)

In [ ]:
def make_y(X, w, noise_fac):
    """ノイズを含めてyの生成を行う．

    Args:
        X (np.array): X
        w (np.array): w
        noise_fac (float): ノイズ値

    Returns:
        _type_: _description_
    """
    y = X * w.reshape(-1)[:, np.newaxis]
    print("add {}*N(0,1) to y".format(noise_fac))
    y += + noise_fac * (np.random.randn(*y.shape)-0.5)*2 
    y = y.reshape(-1)
    print("y.shape=", y.shape)
    return y


g_y = make_y(g_X, g_w, g_noise_fac)


In [ ]:
import os

def save_Xy_file(X, y, w, d, prefix=".", postfix=None):
    """Xとyを保存する．

    ファイル名はw_{postfix}.csvとXy-{d}_{postfix}.csvとなる．

    Args:
        X (np.array): X
        y (np.array): y
        w (float): w
        d (int): 削減割合
        prefix (str, optional): csvを保存するディレクトリ名. Defaults to ".".
        postfix (str, optional): ファイル名につける文字列. Defaults to None.
    """
    df = pd.DataFrame(w.ravel()[:, np.newaxis])
    filename = os.path.join(prefix, "w_{}.csv".format(postfix))
    df.to_csv(filename, index=False, float_format="%.2f", header=False)
    print("file saved to", filename)
    df1 = pd.DataFrame(X.toarray())
    df2 = pd.DataFrame(y)
    df = pd.concat([df1, df2], axis=1)
    filename = os.path.join(prefix, "Xy-{}_{}.csv".format(str(d), postfix))
    df.to_csv(filename, index=False,
              float_format="%.2f", header=False)
    print("file saved to", filename)

if False:
    save_Xy_file(g_X, g_y, g_w, g_d, postfix="L64")


In [ ]:
def L1_reconstruction(X, y, alpha1, fit_intercept, l):
    """Reconstruction with L1 (Lasso) penalization

    y = X w
    の係数wを求める．
    X = proj_operator
    y = proj
    w = rec_l1

    Args:
        X (np.array): descriptor
        y (np.array): target values
        alpha1 (float): hyperparameter of Ridge regression
        fit_intercept (bool): fit intercept or not

    Returns:
        np.array: coefficients of linear function
    """
    rgr_lasso = Lasso(alpha=alpha1, fit_intercept=fit_intercept)

    rgr_lasso.fit(X, y)
    w_l1 = rgr_lasso.coef_.reshape(l, l)
    if fit_intercept:
        print("LASSO intercept", rgr_lasso.intercept_)
    return w_l1


g_w_l1 = L1_reconstruction(g_X, g_y, g_alpha1, g_fit_intercept, g_w.shape[0])
print(g_w_l1.reshape(-1).shape)
g_rmse1 = np.sqrt(mean_squared_error(g_w_l1, g_w))
print("rmse(L1)=", g_rmse1)

In [ ]:
def L2_reconstruction(X, y, alpha2, fit_intercept, l):
    """Reconstruction with L2 (Ridge) penalization

    y = X w
    の係数wを求める．
    X = proj_operator
    y = proj
    w = rec_l1

    Args:
        X (np.array): descriptor
        y (np.array): target values
        alpha1 (float): hyperparameter of Ridge regression
        fit_intercept (bool): fit intercept or not

    Returns:
        np.array: coefficients of linear function
    """
    rgr_ridge = Ridge(alpha=alpha2, fit_intercept=fit_intercept)

    rgr_ridge.fit(X, y)
    w_l2 = rgr_ridge.coef_.reshape(l, l)
    if fit_intercept:
        print("ridge intercept", rgr_ridge.intercept_)
    return w_l2


g_w_l2 = L2_reconstruction(g_X, g_y, g_alpha2, g_fit_intercept, g_w.shape[0])
g_rmse2 = np.sqrt(mean_squared_error(g_w_l2, g_w))
print("rmse(L2)=", g_rmse2)


In [ ]:
def show_images(w, w2, w_title, w2_title, metadata : dict = G_METADATA, 
                titlefontsize=20, tickfontsize=15):
    """再構成した図をならべて表示する．

    Args:
        w (np.array): image 1
        w2 (np.array): image 2
        w_title (str): title 1
        w2_title (str): title 2
        metadata (dict optional): 表示用データ. Defaluts to G_METADATA.
        titlefontsize (int, optional): title font size. Defaults to 20.
        tickfontsize (int, optional): title font size. Defaults to 15.
    """
    plt.figure(figsize=(8, 3.3))
    plt.subplot(121)
    plt.imshow(w, cmap=plt.cm.gray, interpolation='nearest')
    plt.title(w_title, fontsize=titlefontsize)
    plt.axis('off')
    plt.subplot(122)
    plt.imshow(w2, cmap=plt.cm.gray, interpolation='nearest')
    plt.title(w2_title, fontsize=titlefontsize)
    plt.axis('off')
    plt.tight_layout()
    filename = "_".join([metadata["prefix"], metadata["dataname"], 
                         str(metadata["compression"]), str(metadata["noise_fac"]), 
                        w2_title.replace(" ","_"),"image"])+".pdf"
    filepath = os.path.join(metadata["outputdir"],filename)
    print(filepath)
    plt.savefig(filepath)
    plt.show()

    plt.figure(figsize=(8, 3.3))
    plt.subplot(121)
    plt.hist(w.reshape(-1), bins=100)
    plt.title(w_title, fontsize=titlefontsize)
    plt.tick_params(axis = 'x', labelsize =tickfontsize)
    plt.tick_params(axis = 'y', labelsize =tickfontsize)   
    plt.subplot(122)
    plt.hist(w2.reshape(-1), bins=100)
    plt.title(w2_title, fontsize=titlefontsize)
    plt.tick_params(axis = 'x', labelsize =tickfontsize)
    plt.tick_params(axis = 'y', labelsize =tickfontsize)   
    plt.tight_layout()
    filename = "_".join([metadata["prefix"], metadata["dataname"], 
                         str(metadata["compression"]), str(metadata["noise_fac"]), 
                        w2_title.replace(" ","_"),"hist"])+".pdf"
    filepath = os.path.join(metadata["outputdir"],filename)
    print(filepath)
    plt.savefig(filepath)    
    plt.show()

show_images(g_w, g_w_l1, "original", "L1")

In [ ]:
show_images(g_w, g_w_l2, "original", "L2")
